Import Pathway data in BioPAX 3 format to Node and Relationship files for Knowledge Graph Import

[BioPAX – Biological Pathways Exchange
Language
Level 3](
http://www.biopax.org/release/biopax-level3-documentation.pdf)

[PyBioPax documentation](https://pybiopax.readthedocs.io/en/latest/)
[pybiopax repo](https://github.com/indralab/pybiopax)



In [1]:
import os
from pathlib import Path
import pandas as pd
import pybiopax
import pybiopax.biopax
from pybiopax.biopax import *

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns
pd.set_option('display.max_colwidth', None) # don't truncate cells

In [3]:
model = pybiopax.model_from_reactome('R-HSA-162582')
#model = pybiopax.model_from_reactome('R-HSA-5663202')

Processing OWL elements:   0%|          | 0.00/78.2k [00:00<?, ?it/s]

In [4]:
# url = 'https://reactome.org/ReactomeRESTfulAPI/RESTfulWS/biopaxExporter/Level3/162582'
# model = pybiopax.model_from_owl_url(url)

In [5]:
def create_synonym_string(synonyms):
#   Example: [CDC37, CDC37A, HSP90 co-chaperone CDC37] -> CDC37|CDC37A|HSP90 co-chaperone CDC37
    if isinstance(synonyms, list):
        synonyms = '|'.join(synonyms)
    
    return synonyms

In [6]:
def extract_entity_info(component, parent_id, parent_entity, level, relationship):
    entity_type = component.__class__.__name__
    # Prefered database references for proteins and small molecules, default: Reactome
    db_map = {'Protein': 'UniProt', 'SmallMolecule': 'ChEBI'}
    database = db_map.get(entity_type, 'Reactome')
    
    prefix_map = {'Protein': 'uniprot:', 'SmallMolecule': 'chebi:'}
    prefix = prefix_map.get(entity_type, 'reactome:')
    
    synonyms = create_synonym_string(component.name)
    
    if hasattr(component, 'xref'):
        for ref in component.xref:
            if isinstance(ref, UnificationXref):
                if ref.db == database:
                    return {'entity': entity_type, 'id': prefix + ref.id, 'database': ref.db, 'level': level, 
                            'parent_id': parent_id, 'parent_entity': parent_entity, 'relationship': relationship,
                            'name': component.display_name, 'synonyms': synonyms}
    
    if hasattr(component.entity_reference, 'xref'):
        for ref in component.entity_reference.xref:
            if isinstance(ref, UnificationXref):
                # remove CHEBI from id to ensure correct prefix is added.
                ref_id = ref.id
                if 'CHEBI:' in ref_id:
                    ref_id = ref_id.split(':')[1]
                if ref.db == database:       
                    return {'entity': entity_type, 'id': prefix + ref_id, 'database': ref.db, 'level': level, 
                            'parent_id': parent_id, 'parent_entity': parent_entity, 'relationship': relationship,
                            'name': component.display_name, 'synonyms': synonyms}
    
    # no cross-reference found
    return {'entity': entity_type, 'level': level, 
            'parent_id': parent_id, 'parent_entity': parent_entity, 'relationship': relationship,
            'name': component.display_name, 'synonyms': synonyms}

In [7]:
def extract_conversion_info(component, parent_id, parent_entity, level, relationship):
    database = 'Reactome'
    conversion_type = component.__class__.__name__
    synonyms = create_synonym_string(component.name)
    
    if isinstance(component, Conversion):
        if hasattr(component, 'xref'):
            for ref in component.xref:
                if isinstance(ref, UnificationXref):
                    if ref.db == database:
                        return {'entity': conversion_type, 'id': 'reactome:' + ref.id, 'database': database, 'level': level, 
                                'parent_id': parent_id, 'parent_entity': parent_entity, 'relationship': relationship,
                                'name': component.display_name, 'synonyms': synonyms}
    
    return {}

In [8]:
def traverse_pathway(pathway, parent_id, level, rows):
    prefix = '  ' * level

    for pc in pathway.pathway_component:
        synonyms = create_synonym_string(pathway.name)
        display_name = pathway.display_name
        
        for ref in pathway.xref:
            if isinstance(ref, UnificationXref):
                if 'Reactome' in ref.db:
                    reactome_id = 'reactome:' + ref.id
                    
        if isinstance(pc, Pathway):
            ip = {'entity': 'Pathway', 'level': level, 'id': reactome_id, 'database': 'Reactome', 
                  'parent_id': parent_id, 'parent_entity': 'Pathway', 'name': display_name, 'synonyms': synonyms, 'relationship': 'IS_A'}
            rows.append(ip)
            traverse_pathway(pc, reactome_id, level+1, rows)
        elif isinstance(pc, Conversion):
            name = '|'.join(pc.name)
            ic = extract_conversion_info(pc, reactome_id, 'Pathway', level, 'PART_OF')
            rows.append(ic)

            for rx_entity in [pc.left, pc.right]:
                if rx_entity == pc.left:
                    relationship = 'CONSUMED_BY'
                else:
                    relationship = 'PRODUCED_BY'
                for rx_component in rx_entity:             
                    if isinstance(rx_component, Complex):
                        idc = extract_entity_info(rx_component, reactome_id, ic.get('entity'), level, relationship)
                        if len(idc) > 0:
                            rows.append(idc)
                        for component in rx_component.component:
                            idcp = extract_entity_info(component, idc.get('id'), 'Complex', level, 'PART_OF')
                            if len(idcp) > 0:
                                rows.append(idcp)
                    else:
                        idp = extract_entity_info(rx_component, reactome_id, ic.get('entity'), level, relationship)
                        if len(idp) > 0:
                            rows.append(idp)
        else:
            print('Other', type(pc))
            
    return rows

In [9]:
rows = []
parent_id = ''
pw = model.objects.get('Pathway1')
rows = traverse_pathway(pw, parent_id, 1, rows)

In [10]:
df = pd.DataFrame(rows)
df.fillna('', inplace=True)
df.head(25)

,entity,level,id,database,parent_id,parent_entity,name,synonyms,relationship
0,Pathway,1,reactome:R-HSA-162582,Reactome,,Pathway,Signaling Pathways,Signaling Pathways|Signal Transduction,IS_A
1,Pathway,2,reactome:R-HSA-9006934,Reactome,reactome:R-HSA-162582,Pathway,Signaling by Receptor Tyrosine Kinases,Signaling by Receptor Tyrosine Kinases,IS_A
2,BiochemicalReaction,3,reactome:R-HSA-177946,Reactome,reactome:R-HSA-177929,Pathway,Pro-EGF is cleaved to form mature EGF,Pro-EGF is cleaved to form mature EGF,PART_OF
3,Protein,3,uniprot:P01133,UniProt,reactome:R-HSA-177929,BiochemicalReaction,Pro-EGF,Pro-EGF|pro-EGF precursor|Epidermal Growth Factor precursor,CONSUMED_BY
4,Protein,3,uniprot:P01133,UniProt,reactome:R-HSA-177929,BiochemicalReaction,EGF,EGF|Epidermal Growth Factor,PRODUCED_BY
5,BiochemicalReaction,3,reactome:R-HSA-9674531,Reactome,reactome:R-HSA-177929,Pathway,AAMP binds EGFR,AAMP binds EGFR,PART_OF
6,Protein,3,uniprot:Q13685,UniProt,reactome:R-HSA-177929,BiochemicalReaction,AAMP,AAMP|Angio-associated migratory cell protein,CONSUMED_BY
7,Protein,3,uniprot:P00533,UniProt,reactome:R-HSA-177929,BiochemicalReaction,EGFR,EGFR|Epidermal Growth Factor Receptor,CONSUMED_BY
8,Complex,3,reactome:R-HSA-9674862,Reactome,reactome:R-HSA-177929,BiochemicalReaction,EGFR:AAMP,EGFR:AAMP,PRODUCED_BY
9,Protein,3,uniprot:Q13685,UniProt,reactome:R-HSA-9674862,Complex,AAMP,AAMP|Angio-associated migratory cell protein,PART_OF


In [11]:
df['entity'].unique()

array(['Pathway', 'BiochemicalReaction', 'Protein', 'Complex',
       'SmallMolecule', 'PhysicalEntity', 'Rna', 'Dna', 'Degradation'],
      dtype=object)

In [12]:
#df.query('entity == "Rna"')

## Setup Output Directories

In [13]:
NODE_DATA = os.getenv('NODE_DATA', default='../data/nodes/')
RELATIONSHIP_DATA = os.getenv('RELATIONSHIP_DATA', default='../data/relationships/') 

In [14]:
os.makedirs(os.path.join(NODE_DATA), exist_ok=True)
os.makedirs(os.path.join(RELATIONSHIP_DATA), exist_ok=True)

In [15]:
def create_nodes(df, entity):
    # create node dataframe
    df_entity = df.query(f'entity == "{entity}"').copy()
    df_nodes = df_entity[['id', 'name', 'synonyms']].copy()
    df_nodes.drop_duplicates(inplace=True)
    df_nodes = df_nodes[(df_nodes['id'] != '')]
    
    return df_nodes

In [16]:
def create_relationships(df, entity, relationship, parent_entity):
    # create node dataframe
    df_entity = df[(df['entity'] == entity) & (df['parent_entity'] == parent_entity) &
                   (df['relationship'] == relationship)].copy()
    
    # create relationship dataframe
    df_relationships = df_entity[['id', 'parent_id']].copy()
    df_relationships.rename(columns={'id': 'from', 'parent_id': 'to'}, inplace=True)
    df_relationships.drop_duplicates(inplace=True)
    df_relationships = df_relationships[(df_relationships['from'] != '') & (df_relationships['to'] != '')]
    
    return df_relationships

In [17]:
def save_nodes_and_relationships(key, df):
    nodes = create_nodes(df, key[0])
    nodes.to_csv(os.path.join(NODE_DATA, f'{key[0]}.csv'), index=False)
    
    print('Saving:', f'{key[0]}-{key[1]}-{key[2]}.csv')
    relationships = create_relationships(df, key[0], key[1], key[2])
    relationships.to_csv(os.path.join(RELATIONSHIP_DATA, f'{key[0]}-{key[1]}-{key[2]}.csv'), index=False)

In [18]:
groups = df.groupby(['entity','relationship', 'parent_entity'])

In [19]:
entities = ['Pathway', 'BiochemicalReaction', 'Complex', 'Protein', 'SmallMolecule']
for key, group in groups:
    if key[0] in entities and key[2] in entities: 
        print(key, group.shape[0])
        save_nodes_and_relationships(key, group)

('BiochemicalReaction', 'PART_OF', 'Pathway') 2596
Saving: BiochemicalReaction-PART_OF-Pathway.csv
('Complex', 'CONSUMED_BY', 'BiochemicalReaction') 1916
Saving: Complex-CONSUMED_BY-BiochemicalReaction.csv
('Complex', 'PART_OF', 'Complex') 2785
Saving: Complex-PART_OF-Complex.csv
('Complex', 'PRODUCED_BY', 'BiochemicalReaction') 2147
Saving: Complex-PRODUCED_BY-BiochemicalReaction.csv
('Pathway', 'IS_A', 'Pathway') 420
Saving: Pathway-IS_A-Pathway.csv
('Protein', 'CONSUMED_BY', 'BiochemicalReaction') 1583
Saving: Protein-CONSUMED_BY-BiochemicalReaction.csv
('Protein', 'PART_OF', 'Complex') 4367
Saving: Protein-PART_OF-Complex.csv
('Protein', 'PRODUCED_BY', 'BiochemicalReaction') 621
Saving: Protein-PRODUCED_BY-BiochemicalReaction.csv
('SmallMolecule', 'CONSUMED_BY', 'BiochemicalReaction') 954
Saving: SmallMolecule-CONSUMED_BY-BiochemicalReaction.csv
('SmallMolecule', 'PART_OF', 'Complex') 740
Saving: SmallMolecule-PART_OF-Complex.csv
('SmallMolecule', 'PRODUCED_BY', 'BiochemicalReactio